In [1]:
# !py -m pip install pycalphad
# !py -m pip install numpy
# !py -m pip install tqdm
# !py -m pip install ipywidgets

In [2]:
from pycalphad import Database, equilibrium, variables as v
# from pycalphad.core.utils import filter_phases
import random
import numpy as np
import time
from tqdm import tqdm
# from tqdm.auto import tqdm

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
# TODO: check 'PM' param
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()

c:\Users\Dmitry\AppData\Local\Programs\Python\Python310\lib\site-packages\pycalphad\io\tdb.py:925: UserWarning: The type definition character `K` was defined in the following phases: ['FCC_A1', 'FCC_L12'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
c:\Users\Dmitry\AppData\Local\Programs\Python\Python310\lib\site-packages\pycalphad\io\tdb.py:925: UserWarning: The type definition character `S` was defined in the following phases: ['FCC_A1'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
c:\Users\Dmitry\AppData\Local\Programs\Python\Python310\lib\site-packages\pycalphad\io\tdb.py:925: UserWarning: The type definition character `P` was defined in the following phases: ['FCC_A1', 'FCC_4SL', 'HCP_A3', 'HCP_4SL'], but no corresponding TYPE_DEFINITION line wa

In [3]:
def getPhases(int_mask, all_phases=all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int_mask & (1 << i)) > 0]

def phasesToMask(phases, all_phases=all_phases):
    phases.sort()
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)


def phaseToMask(phase, all_phases=all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

def compToMask(components, all_components=all_components):
    components.sort()
    mask = [1 if comp in components else 0 for comp in all_components][::-1]
    return int(''.join(map(str, mask)), 2)

def getComponents(int_mask, all_components=all_components):
    return [all_components[i] for i in range(len(all_components)) if (int_mask & (1 << i)) > 0]


In [24]:
import pandas as pd

df = pd.read_csv('DataSets\equilibrium_result.csv',delimiter = ';')
# TODO: Parse phases
# TODO: Parse amounts
# TODO: get list of functions for phases
df['Components'] = df['Components'].apply(getComponents)
# df['phases'] = df['phases'].astype('long')
print(df.dtypes)
df

iter                 int64
T                    int64
amounts             object
Components          object
phases              object
ellapsed_time      float64
possible_phases     object
P                    int64
Error               object
dtype: object


,iter,T,amounts,Components,phases,ellapsed_time,possible_phases,P,Error
0,0,430,"[0.08967186972979238, 0.17727390265453108, 0.2...","[BI, C, MG, OS, PR, SM, TC, ZR]",6661076604585215412432153809513014744713767838...,13.438566,8558039834531271110329700367419095489801340099...,101325,NaN
1,1,2713,"[0.21780216569867217, 0.42648142933184324, 0.0...","[BI, C, MG, OS, PR, SM, TC, ZR]",6084483977492414823814226985991697806408637340...,13.299496,8558039834531271110329700367419095489801340099...,101325,NaN
2,2,718,"[0.10566681910467296, 0.10264780093722332, 0.4...","[BI, C, MG, OS, PR, SM, TC, ZR]",6863899865208687544835263299417144832869651850...,13.746437,8558039834531271110329700367419095489801340099...,101325,NaN
3,0,430,"[0.08967186972979238, 0.17727390265453108, 0.2...","[BI, C, MG, OS, PR, SM, TC, ZR]",6661076604585215412432153809513014744713767838...,13.784137,8558039834531271110329700367419095489801340099...,101325,NaN
4,1,2713,"[0.21780216569867217, 0.42648142933184324, 0.0...","[BI, C, MG, OS, PR, SM, TC, ZR]",6084483977492414823814226985991697806408637340...,13.638592,8558039834531271110329700367419095489801340099...,101325,NaN
...,...,...,...,...,...,...,...,...,...
776,14,1039,"[0.31437635253989943, 0.23835318096726754, 0.1...","[AL, BA, BE, CR, K, NB, SM, ZN]",1415180343103247007809130306480600982765741208...,34.693300,7780036213316397304961794752084977965231686877...,101325,NaN
777,15,1397,"[0.331769128304529, 0.209815800507963, 0.14771...","[AL, BA, BE, CR, K, NB, SM, ZN]",6084483977492414823814226985991697806408637340...,24.870340,7780036213316397304961794752084977965231686877...,101325,NaN
778,16,1792,"[0.1887902482834592, 0.12211218435422921, 0.27...","[AL, BA, BE, CR, K, NB, SM, ZN]",6084483977492414823814226985991697806408637340...,24.988712,7780036213316397304961794752084977965231686877...,101325,NaN
779,17,2896,"[0.12035822990623389, 0.17373151391804537, 0.2...","[AL, BA, BE, CR, K, NB, SM, ZN]",6084483977492414823814226985991697806408637340...,24.005069,7780036213316397304961794752084977965231686877...,101325,NaN
